In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/housing-prices-competition-for-kaggle-learn-users/train.csv
/kaggle/input/housing-prices-competition-for-kaggle-learn-users/test.csv


In [2]:
import pandas as pd
X_train = pd.read_csv('../input/housing-prices-competition-for-kaggle-learn-users/train.csv', index_col='Id') 
X_test = pd.read_csv('../input/housing-prices-competition-for-kaggle-learn-users/test.csv', index_col='Id')


In [3]:
#Q1
X_train.dropna(axis=0, subset=["SalePrice"], inplace = True)

In [4]:
#Q2
y_train = X_train.SalePrice
X_train.drop(['SalePrice'], axis = 1, inplace = True)
X_train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [7]:
#Q3
low_cardinality_cols = [cname for cname in X_train.columns if X_train[cname].nunique() < 10 and 
                        X_train[cname].dtype == "object"]
numeric_cols = [cname for cname in X_train.columns if X_train[cname].dtype in ['int64', 'float64']]

my_cols = low_cardinality_cols + numeric_cols

X_train_selected = X_train[my_cols].copy()
X_test_selected = X_test[my_cols].copy()

X_train_selected = pd.get_dummies(X_train_selected)
X_test_selected = pd.get_dummies(X_test_selected)

X_train_selected, X_test_selected = X_train_selected.align(X_test_selected, join='left', axis=1)

In [8]:
#Q4
from sklearn.impute import SimpleImputer
imputer = SimpleImputer()
X_train_imputed = pd.DataFrame(imputer.fit_transform(X_train_selected))
X_test_imputed = pd.DataFrame(imputer.transform(X_test_selected))

X_train_imputed.columns = X_train_selected.columns
X_test_imputed.columns = X_test_selected.columns

In [9]:
#Q5
from sklearn.model_selection import train_test_split

X_train_imputed_2, X_valid_imputed, y_trainm, y_validm = train_test_split(X_train_imputed, y_train,train_size=0.8, test_size=0.2, random_state=0)


In [10]:
#Q6
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

model = XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=-1, random_state=0)
model.fit(X_train_imputed_2, y_trainm)
preds_test = model.predict(X_valid_imputed)
mae = mean_absolute_error(preds_test, y_validm)
print(mae)

17090.218255029966


MAE: 17090.218255029966

In [12]:
#Q7
from sklearn.model_selection import cross_val_score 
def score_dataset(X_train, y_train, n_estimators):
    model = XGBRegressor(n_estimators=n_estimators, learning_rate=0.05, random_state=0, n_jobs=-1)
    model.fit(X_train, y_train)
    maes = -1 * cross_val_score(model, X_train, y_train, cv=3, scoring='neg_mean_absolute_error') 
    mae = maes.mean() 
    print("Cross-validation MAE:", mae) 
    
    return mae

In [13]:
#Q8
n_estimators_values = [100, 500, 1000, 1500]

for n_estimators in n_estimators_values: 
    
    mae = score_dataset(X_train_imputed, y_train, n_estimators)
    
    if (n_estimators == n_estimators_values[0]):
        best_mae = mae 
        best_n_estimators = n_estimators
    
    elif (mae < best_mae):
        best_mae = mae 
        best_n_estimators = n_estimators
    
    print("MAE:", mae)

print("Best n_estimators:", best_n_estimators) 
print("Best MAE:", best_mae) 

Cross-validation MAE: 16687.61763912127
MAE: 16687.61763912127
Cross-validation MAE: 16151.134240169386
MAE: 16151.134240169386
Cross-validation MAE: 16154.324909588824
MAE: 16154.324909588824
Cross-validation MAE: 16158.865987319174
MAE: 16158.865987319174
Best n_estimators: 500
Best MAE: 16151.134240169386


Best n_estimators: 500

Best MAE: 16151.134240169386

In [14]:
#Q9
n_estimators = best_n_estimators
model = XGBRegressor(n_estimators=n_estimators, learning_rate=0.05, n_jobs=-1, random_state=0)
model.fit(X_train_imputed, y_train)
preds_test = model.predict(X_test_imputed)

In [15]:
#Q10
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)

MY LEADERBOARD IS 15040.9925